# <center> Predicting Donors Choose</center>

--------

# Introduction

## Imports

https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html

In [4]:
import gc
import numpy as np
import pandas as pd
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm_notebook

In [5]:
#import lightgbm as lgb

In [6]:
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import re

In [7]:
#I would rather not run the while dataset on my laptop
# so kaggle determines if I am also evaluating the kaggle test set
kaggle = False

#sim refers to if data simulating variable situations is being generated & tested on. 
sim = False

#Kaggle & Sim use 'test' so dont use both

In [8]:
train = pd.read_csv('./Input/train.csv', low_memory=False, index_col='id')
if kaggle: 
    if sim == False:
        test = pd.read_csv('./Input/test.csv', low_memory=False, index_col='id')

res = pd.read_csv('./Input/resources.csv', low_memory=False, index_col='id')
train['project_submitted_datetime'] = pd.to_datetime(train['project_submitted_datetime'])

In [9]:
def make_sim(n='p039565'):
    """
    Makes a semi-brute forced dataset based off the entry n, with variable:
    Months (jan-dec), 
    Pronouns (not teacher or dr), 
    Previous Entries(0:30),
    to see what combination may have a higher success rate
    """
    test = pd.DataFrame(train.loc[n]).transpose()
    del test['project_is_approved']
    
    change_dict = {"prefix":['Mrs.','Ms.','Mr.'],
        "date":['2017-01-26','2017-02-26','2017-03-26','2017-04-26','2017-05-26','2017-06-26',
        '2017-07-26','2017-08-26','2017-09-26','2017-10-26','2017-11-26','2017-12-26',],
        "prev" : [x for x in range(0,10)]}

    for i in range(100):
        test.loc[n+str(i)] = test.loc[n].transpose()
        test.set_value(n+str(i),'teacher_prefix',np.random.choice(change_dict['prefix']))
        test.set_value(n+str(i),'project_submitted_datetime',np.random.choice(change_dict['date']))
        test.set_value(n+str(i),'teacher_number_of_previously_posted_projects',np.random.choice(change_dict['prev']))
        
    test.index.rename = 'id'
    return(test)

In [10]:
#making a dataset which has variable situations that 'p039654' may be under
if sim:
    test = make_sim('p039565')
    train = train.drop('p039565',axis=0)
    del make_sim

In [11]:
train = train[:10000]

## Preprocessing

### 1.1 Resource Intergration
Here we evaluate how much each project/proposal will cost and/or how big they are

In [12]:
res['cost'] = res['quantity'] * res['price']
res_agg = res.groupby('id').agg({'description': ['nunique'], 'quantity': ['sum'], 'cost': ['mean', 'sum']})
res_agg.columns = ['unique_items', 'total_quantity', 'mean_cost', 'total_cost']
res_agg.reset_index(inplace=True)

#description was dropped because the description of the project should not have an effect on its liklihood of success

In [13]:
train = train.merge(res_agg, left_index=True, right_on='id')
if kaggle: 
    if sim : 
        for i in res_agg.columns:
            if i != 'id':
                test[i] = res_agg[res_agg['id']=='p039565'].drop('id',axis=1)[i].values[0]
    else:
        test =  test.merge(res_agg, left_index=True, right_on='id')

del res_agg
del res

### 1.2 Preprocessing of features

#### EDA

In [14]:
"""
train_resource["project_submitted_datetime"] = pd.to_datetime(train_resource["project_submitted_datetime"])
train_resource["month_created"] = train_resource["project_submitted_datetime"].dt.month
loan = train_resource.groupby(['school_state', 'month_created'])['price'].mean().unstack()
loan = loan.sort_values([3], ascending=False)
f, ax = plt.subplots(figsize=(15, 20)) 
loan = loan.fillna(0)
temp = sns.heatmap(loan, cmap='Reds')
plt.show()
"""

'\ntrain_resource["project_submitted_datetime"] = pd.to_datetime(train_resource["project_submitted_datetime"])\ntrain_resource["month_created"] = train_resource["project_submitted_datetime"].dt.month\nloan = train_resource.groupby([\'school_state\', \'month_created\'])[\'price\'].mean().unstack()\nloan = loan.sort_values([3], ascending=False)\nf, ax = plt.subplots(figsize=(15, 20)) \nloan = loan.fillna(0)\ntemp = sns.heatmap(loan, cmap=\'Reds\')\nplt.show()\n'

In [15]:
#gen_mon = train.groupby(['datetime_month','gender'])['project_is_approved'].mean().reset_index()
#prev_mon = train.groupby('teacher_number_of_previously_posted_projects')['project_is_approved'].mean().reset_index()

#bins = pd.cut(train['teacher_number_of_previously_posted_projects'], [0, 25, 50, 75, 100, 125, 150])
#pros = train.groupby(bins)['project_is_approved'].agg('mean')

In [16]:
#sns.barplot(pros.index,pros)
#plt.title('Frequent Applicants')
#plt.xlabel("Teacher's Previous Postings")
#plt.ylabel("Mean Approvals")

In [17]:
#train.project_is_approved.value_counts()

In [18]:
#sns.lmplot(x="datetime_month", y="project_is_approved", hue="gender", 
#           data=gen_mon[gen_mon['gender']!='Unk'], 
#           order=3,ci=80)
#plt.title('Approval rate, by month, by gender (ci:90)')

In [19]:
#del gen_mon
#del prev_mon

In [20]:
gc.collect()

375

#### Cat Preprocessing

There are 51 States, because of 50 + DC

In [21]:
train.isnull().sum()[train.isnull().sum()>0]

project_essay_3    9648
project_essay_4    9648
dtype: int64

In [22]:
train.teacher_prefix[train.teacher_prefix.isnull()] = 'Teacher'
try: 
    test.teacher_prefix[test.teacher_prefix.isnull()] = 'Teacher'
except:
    pass

/Users/Nahel/anaconda3/envs/NN/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


fill na did not work

In [23]:
def date_prep(train):
    train['datetime_year'] = train['project_submitted_datetime'].dt.year
    train['datetime_month'] = train['project_submitted_datetime'].dt.month
    return(train)

In [24]:
def gender_features(train):
    conditions = [(train.teacher_prefix == 'Mr.'), 
                  (train.teacher_prefix == 'Mrs.') | (train.teacher_prefix == 'Ms.')]
    choices = ['Male', 'Female']
    train['gender'] = np.select(conditions, choices, default='Unk')
    return(train)

In [25]:
date_prep(train)
if kaggle: date_prep(test)

train = gender_features(train)
if kaggle: test = gender_features(test)
    
del train['project_submitted_datetime']
del train['project_subject_subcategories']
del date_prep, gender_features

In [26]:
gc.collect()

45

##### Encoding labels

In [27]:
# Because of memory issues, it made more sense to encode everything as a string rather than dummies
cols  = ['gender',
        'teacher_prefix', 
        'school_state',
        'project_grade_category']

for c in tqdm_notebook(cols):
    dummies = pd.get_dummies(train[c])
    train = train.join(dummies)
    del train[c]

HBox(children=(IntProgress(value=0, max=4), HTML(value='')))

#### Num Preprocessing

In [28]:
num_features  = ['teacher_number_of_previously_posted_projects','total_quantity', 'mean_cost', 'total_cost',]

In [29]:
SS = StandardScaler()
train[num_features] = SS.fit_transform(train[num_features])
if kaggle: test[num_features] = SS.fit_transform(test[num_features])

In [30]:
del num_features, StandardScaler, SS

In [31]:
gc.collect()

112

#### Text Preprocessing

In [32]:
other_stopwords = [x for x in 'abcdefghijklmnopqrstuvwxyz']
for j in ['student','students','education',]:
    other_stopwords.append(j)

In [33]:
#import gensim
from nltk.corpus import stopwords
def scrub(text):
    text = text.strip().lower()
    text = re.sub('\W+',' ', text)    
    text = re.sub(r'_', ' ', text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\:', ' ', text)
    text = re.sub(r'\+', ' ', text)
    text = re.sub(r'\=', ' ', text)
    text = re.sub(r'(\")', ' ', text)
    text = re.sub(r'(\r)', ' ', text)
    text = re.sub(r'(\n)', ' ', text)
    text = re.sub(r'(\\)', ' ', text)
    text = re.sub('n t ','n\'t ', text)
    text = re.sub(' re ',' are ', text)
    text = re.sub(r'(\r\n)', ' ', text)
    text = re.sub(r'\"\"\"\"', ' ', text)
    text = re.sub(' i m ',' i\'m ', text)
    return(text)
def swords(text):
    text = ' '.join([word for word in text.split() if word not in (other_stopwords + stopwords.words("english"))])
    return(text)

##### Project Category Preprocessing

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

In [35]:
ctv = CountVectorizer()
project_cats = train.project_subject_categories.apply(lambda x: scrub(x))
x = ctv.fit_transform(project_cats)
train = train.reset_index

In [36]:
train = train().merge(pd.DataFrame(x.toarray(), 
                         columns=["subject_cat_"+nm for nm in ctv.get_feature_names()])
            ,left_index=True,
            right_index=True)
del ctv, x ,project_cats, train['project_subject_categories']

##### Essay Preprocessing

  
#### Before May 17th, 2016:

- project_essay_1: "Introduce us to your classroom"
- project_essay_2: "Tell us more about your students"
- project_essay_3: "Describe how your students will use the materials you're requesting"
- project_essay_4: "Close by sharing why your project will make a difference"

#### May 17th, 2016 and beyond:

- project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."
- project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"

#### Plan
- Combine essay_1 and essay_2 before May 17th to make "student_description" and use essay_1 after May 17th directly
- Combine essay_3 and essay_4 before May 17th to make "project_description" and use essay_2 after May 17th directly

In [37]:
def essay_convert(train):
    # Making the First essay : student_description
    train['student_description']=train['project_essay_1']
    train.loc[train.project_essay_3.notnull(),'student_description']=train.loc[train.project_essay_3.notnull(),'project_essay_1']+train.loc[train.project_essay_3.notnull(),'project_essay_2']

    # Making the second essay : project_description
    train['project_description']=train['project_essay_2']
    train.loc[train.project_essay_3.notnull(),'project_description']=train.loc[train.project_essay_3.notnull(),'project_essay_3']+train.loc[train.project_essay_3.notnull(),'project_essay_4']

    # Removing original essays
    del train['project_essay_1']
    del train['project_essay_2']
    del train['project_essay_3']
    del train['project_essay_4']
    return(train)

In [38]:
essay_convert(train)
if kaggle: essay_convert(test)
gc.collect()

del essay_convert

##### Scrub Text Features

In [39]:
text_features = ['project_title', 'project_resource_summary',
                'project_description', 'student_description']

In [40]:
for j in tqdm_notebook(text_features):
    n_col = 'processed_'+j
    train[n_col] = train[j].apply(lambda x: scrub(x)).apply(lambda x: swords(x))
    if kaggle: test[n_col] = test[j].apply(lambda x: scrub(x)).apply(lambda x: swords(x))

gc.collect()
for i in text_features:
    del train[i]
    if kaggle: del test[i]

del stopwords, other_stopwords, text_features

HBox(children=(IntProgress(value=0, max=4), HTML(value='')))

KeyboardInterrupt: 

In [ ]:
gc.collect()

In [ ]:
train.columns

## Modeling

### Tfidf & X,y Assignment

In [ ]:
gc.collect()

In [ ]:
cols = [
    'processed_project_title',
    'processed_project_resource_summary', 
    'processed_project_description',
    'processed_student_description']

In [ ]:
for i in cols:
    print("Average length in {} is {} words".format(i,str(round(train[i].str.len().mean()))))

In [ ]:
n_features = [
    100, 
    200, 
    1500,
    1500]

In [ ]:
gc.collect()

In [ ]:
for c_i, c in tqdm_notebook(enumerate(cols)):
    tfidf = TfidfVectorizer(
        ngram_range=(1,3),
        max_features=n_features[c_i])

    tfidf.fit(train[c])

#    tfidf_train2 = tfidf.transform(train[c])

    tfidf_train = np.array(tfidf.transform(train[c].values).toarray(), dtype=np.float16)
    for i in range(n_features[c_i]):
        train[c + '_tfidf_' + str(i)] = tfidf_train[:, i]
    if kaggle:
        tfidf_test = np.array(tfidf.transform(test[c].values).toarray(), dtype=np.float16)
        for i in range(n_features[c_i]):
            test[c + '_tfidf_' + str(i)] = tfidf_test[:, i]

for i in cols:
    del train[i]
    if kaggle: del test[i]

In [ ]:
#did this a while ago
#train = train.reset_index()
del train['index']

In [ ]:
train.columns[1:50]

In [ ]:
drop_cols = ['project_is_approved','id','teacher_id']

X = train.drop(drop_cols, axis=1)
y = train['project_is_approved']
feature_names = list(X.columns)
del train

In [ ]:
#if running demo, uncomment this and change test['id'].vaues -> test['index'].values
#test = test.reset_index()

In [ ]:
X_test = test.drop(drop_cols, axis=1, errors='ignore')
id_test = test['id'].values

### Back to Neural Networks!!!

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(X,y)

In [129]:
import keras
from keras import regularizers, optimizers

from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'keras'

In [ ]:
adam = optimizers.Adam(lr=0.01)
l2 = regularizers.l2(0.1)

In [ ]:
model = Sequential()
model.add(Dense(528, 
                activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(256, 
                activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(128, 
                activation='relu'))
model.add(Dense(1, 
                activation='sigmoid'))

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              metrics=['accuracy'],
             optimizer = adam)

early_stop = EarlyStopping(monitor='val_loss', min_delta=1, patience=10)

callbacks_list = [early_stop]

In [ ]:
from sklearn.metrics import roc_auc_score
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))])

### Grid Search over LGBM for best params without training

In [125]:
model = lgb.LGBMClassifier( 
    boosting_type="gbdt",
    is_unbalance=True, 
    random_state=10,
    bagging_freq=5, 
    learning_rate=0.025,
    min_child_samples=3,
    verbose=1)

In [126]:
params_opt = {'n_estimators':  [50],
             'max_depth': range(15,16),
             'feature_fraction': [x / 1000.0 for x in range(825,900,25)],
             'bagging_fraction': [x / 1000.0 for x in range(825,900,25)],
             'num_leaves':range(20,30,5),
             'lambda_l2' : [0.1,0.2,0.3]}

In [127]:
from sklearn.model_selection import RandomizedSearchCV

In [128]:
rs = RandomizedSearchCV(
    model, 
    params_opt, 
    n_iter=3,
    scoring='roc_auc',
    verbose=1,
    cv=3)
rs.fit(X,y)
rs.best_score_

Fitting 3 folds for each of 3 candidates, totalling 9 fits


KeyboardInterrupt: 

In [ ]:
train.head()

### Training Best LGBM

In [121]:
# Build the model
cnt = 0
n_splits = 5
n_repeats = 1
kf = RepeatedKFold(
    n_splits=n_splits, 
    n_repeats=n_repeats, 
    random_state=42)

for train_index, valid_index in kf.split(X):
    print('Fold {}/{}'.format(cnt + 1, n_splits))
    params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'max_depth': 10,
        'num_leaves': 10,
        'learning_rate': 0.025,
        'feature_fraction': 0.85,
        'bagging_fraction': 0.85,
        'bagging_freq': 5,
        'verbose': 0,
        'min_gain_to_split': 5,
        'lambda_l2':0.5}  

    lgb_train = lgb.Dataset(
        X.loc[train_index], 
        y.loc[train_index], 
        feature_name=feature_names)

    lgb_valid = lgb.Dataset(
        X.loc[valid_index], 
        y.loc[valid_index])

    model = lgb.train(
        params, lgb_train,
        num_boost_round=10000,
        valid_sets=[lgb_train, lgb_valid],
        early_stopping_rounds=100,
        verbose_eval=10,)

    if cnt == 0:
        importance = model.feature_importance()
        model_fnames = model.feature_name()
        tuples = sorted(zip(model_fnames, importance), key=lambda x: x[1])[::-1]
        tuples = [x for x in tuples if x[1] > 0]
        print('Important features:')
        for i in range(60):
            if i < len(tuples):
                print(tuples[i])
            else:
                break
    
    cnt = cnt+1

Fold 1/5
Training until validation scores don't improve for 100 rounds.
[10]	training's auc: 0.708112	valid_1's auc: 0.655858
[20]	training's auc: 0.725977	valid_1's auc: 0.660433
[30]	training's auc: 0.73155	valid_1's auc: 0.667443
[40]	training's auc: 0.737278	valid_1's auc: 0.667594
[50]	training's auc: 0.741838	valid_1's auc: 0.669593
[60]	training's auc: 0.757168	valid_1's auc: 0.680699
[70]	training's auc: 0.762158	valid_1's auc: 0.680982
[80]	training's auc: 0.770717	valid_1's auc: 0.684989
[90]	training's auc: 0.780088	valid_1's auc: 0.689824
[100]	training's auc: 0.786157	valid_1's auc: 0.69112
[110]	training's auc: 0.793819	valid_1's auc: 0.692828
[120]	training's auc: 0.800645	valid_1's auc: 0.695959
[130]	training's auc: 0.808308	valid_1's auc: 0.698004
[140]	training's auc: 0.813397	valid_1's auc: 0.703586
[150]	training's auc: 0.820361	valid_1's auc: 0.705336
[160]	training's auc: 0.827057	valid_1's auc: 0.70467
[170]	training's auc: 0.834051	valid_1's auc: 0.703644
[180]

KeyboardInterrupt: 

In [ ]:
#p = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
idf = tfidf.idf_
idf_map = dict(zip(tfidf.get_feature_names(), idf))

In [ ]:
tfidf.get_feature_names()[75]

In [ ]:
tfidf.idf_[75]

In [ ]:
#idf_map

In [ ]:
submit = pd.DataFrame({'id':id_test,'pred':p})

In [ ]:
#submit.sort_values('pred',axis=0,)